In [1]:
import os
import psycopg2

from dotenv import load_dotenv

from pgvector.psycopg2 import register_vector

from utils import api, db, prompt, prompting

load_dotenv()

conn = psycopg2.connect(os.getenv("POSTGRES_CONFIG"))
register_vector(conn)
cursor = conn.cursor()

In [2]:
resource_id = db.get_resource_id(cursor, "Test BPK - Fact new")
print(resource_id)

9283f7c0-da94-45da-84a6-cd8b9df07dcb


In [3]:
query = "Di bidang apakah Anggota 3 bertugas?"
resource_ids = [resource_id]
knowledges = db.get_retrieved_knowledge(cursor, query, resource_ids, 10)
retrieved = [k[1] for k in knowledges]

In [9]:
chat_prompt = prompt.get_chat_prompt(query, retrieved, memory_max_tokens=500, config="context")
print(chat_prompt)

[BOT]
--- SYSTEM INFO:
Today's Time: Friday, 05 January 2024 06:18AM
--- BOT INSTRUCTION

--- USER INFO:
{"name":"John","email":"john@feedloop.ai"}

--- KNOWLEDGE:
1. Test BPK - Fact new.pdf
Doc URL: https://platform-staging-api.feedloop.ai/download/603f9138-2723-4b28-bc56-c32190521e64/Test BPK - Fact new.pdf
Context 1: Anggota IV BPK RI bertanggung jawab terhadap kementerian yang terkait dengan energi dan sumber daya alam.
Facts from Context 1:
- Anggota IV BPK RI memiliki obyek tugas pada kementerian yang terkait bidang energi dan sumber daya alam.
Context 2: Anggota III BPK RI bertanggung jawab terhadap kementerian yang terkait dengan Lembaga Tinggi Negara dan kesejahteraan sosial.
Facts from Context 2:
- Anggota III BPK RI memiliki obyek tugas pada kementerian yang terkait bidang Lembaga Tinggi Negara dan kesejahteraan sosial.
Context 3: Anggota V BPK RI bertanggung jawab terhadap kementerian yang terkait dengan Dalam Negeri, Keagamaan, dan Pemerintah Daerah.
Facts from Context 3:


In [5]:
answer = api.get_completions_gpt4([
        {
        "role": "user",
        "content": chat_prompt,
        }
    ], temp=0)

In [ ]:
response = answer.choices[0].message.content
print(response)